In [1]:
import pandas as pd 
import numpy as np
from eppy import modeleditor
from eppy.modeleditor import IDF
from pyDOE3 import lhs ## pyDOE3 is the new version of pyDOE
from scipy.stats import uniform ## to map the generated sample to a uniform distribution
import time 
import pyarrow as pa ## Not necessary. You can store the data in a csv file, but parquets are better 
import pyarrow.parquet as pq 

## IDD, IDF and EPW

In [2]:
idd="C:\EnergyPlusV23-2-0/Energy+.idd"
IDF.setiddname(idd)

In [3]:
idf_loc='C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\FinalDOE\\DOE_V10.idf'
epw_loc='C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\FinalDOE\\CAN_ON_OTTAWA-INTL-A_6106001_CWEC.epw'

In [4]:
idf=IDF(idf_loc,epw_loc)

In [6]:
idf.idfobjects['OutputControl:Files']

[
OutputControl:Files,
    Yes,                      !- Output CSV
    Yes,                      !- Output MTR
    Yes,                      !- Output ESO
    Yes,                      !- Output EIO
    Yes,                      !- Output Tabular
    Yes,                      !- Output SQLite
    Yes,                      !- Output JSON
    Yes,                      !- Output AUDIT
    No,                       !- Output Zone Sizing
    No,                       !- Output System Sizing
    No,                       !- Output DXF
    Yes,                      !- Output BND
    Yes,                      !- Output RDD
    Yes,                      !- Output MDD
    Yes,                      !- Output MTD
    Yes,                      !- Output END
    Yes,                      !- Output SHD
    Yes,                      !- Output DFS
    Yes,                      !- Output GLHE
    No,                       !- Output DelightIn
    No,                       !- Output DelightELdmp
    No,  

In [5]:
idf.run()


C:\EnergyPlusV23-2-0\energyplus.exe --weather C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\FinalDOE\CAN_ON_OTTAWA-INTL-A_6106001_CWEC.epw --output-directory C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\Notebooks --idd C:\EnergyPlusV23-2-0\Energy+.idd C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\FinalDOE\DOE_V10_e01fd7.idf



## Inputs and Ranges

In [5]:
U_wall_min= 0.126
U_wall_max= 0.264
U_roof_min= 0.09
U_roof_max= 0.152

In [6]:
## convert U_values to R_values
R_wall_max= 1/U_wall_min
R_wall_min= 1/U_wall_max
R_roof_min= 1/U_roof_max
R_roof_max= 1/U_roof_min

In [8]:
R_wall_max

7.936507936507937

In [9]:
R_wall_min

3.7878787878787876

In [7]:
print(R_roof_min)
print(R_roof_max)

6.578947368421053
11.11111111111111


In [9]:
## Wall min and max R_insulation 
WR1= idf.idfobjects['Material'][-6].Thickness/idf.idfobjects['Material'][-6].Conductivity ##1IN Stucco
WR2= idf.idfobjects['Material'][-4].Thickness/idf.idfobjects['Material'][-4].Conductivity ##8IN CONCRETE HW RefBldg
WR_ins_min= R_wall_min-WR1-WR2
WR_ins_max= R_wall_max-WR1-WR2

In [ ]:
WR_ins_min

In [ ]:
WR_ins_max

In [10]:
### Roof min and max R_insulation 
RR1= idf.idfobjects['Material'][-1].Thickness/idf.idfobjects['Material'][-1].Conductivity ##BETB Asphalt Cover Board and Membrane
RR2= idf.idfobjects['Material'][-2].Thickness/idf.idfobjects['Material'][-2].Conductivity ##BETB Gypsum Board
RR3= idf.idfobjects['Material'][-3].Thickness/idf.idfobjects['Material'][-3].Conductivity ##BETB Steel Deck
R_ins_min= R_roof_min-RR1-RR2-RR3 ## Lower bound
R_ins_max= R_roof_max-RR1-RR2-RR3 ## Upper bound

In [ ]:
R_ins_max

In [ ]:
R_ins_min

In [14]:
idf.idfobjects['Construction'][0].Name="Elin"

In [15]:
idf.idfobjects['Construction'][0].Name

'Elin'

In [ ]:
idf.idfobjects['Construction'][-3]

In [ ]:
idf.idfobjects['Material:NoMass']

In [ ]:
idf.idfobjects['Material'][-6]

In [ ]:
idf.idfobjects['Material'][-1]

In [ ]:
idf.idfobjects['Material'][-1].Conductivity

In [ ]:
idf.idfobjects['Material'][-1].Thickness

In [ ]:
idf.idfobjects['Material'][-2]

In [ ]:
idf.idfobjects['Material'][-3]

## Inputs, Ranges and Distribution 

In [10]:
ranges={
    'X1':{'min':WR_ins_min,'max':WR_ins_max}, ## Wall Insulation R_value
    'X2':{'min':R_ins_min,'max':R_ins_max}, ## Roof Insulation R_value
    'X3':{'min':1.1511,'max':1.903}, ## Window U_value
    'X4':{'min':0,'max':1}, ## SHGC
    'X5':{'min':2.407,'max':4.598}, ## COP-High Speed
    'X6':{'min':2.407,'max':4.598}, ## COP-Low Speed
    'X7':{'min':16,'max':22}, ## Occupied Heating SP
    'X8':{'min':16,'max':22}, ## UNOccupied Heating SP 
    'X9':{'min':22.01,'max':28}, ## Occupied Cooling SP
    'X10':{'min':22.01,'max':28} ## UNOccupied Cooling SP
}

In [ ]:
len(ranges)

In [ ]:
## Refer to: https://pydoe3.readthedocs.io/en/latest/randomized.html#latin-hypercube

In [11]:
n_samples= 500 ## Number of samples 

In [12]:
lhs_samples= lhs(len(ranges), samples=n_samples, criterion='maximin', random_state=42)

In [ ]:
lhs_samples

In [13]:
# Map LHS samples to the specified ranges
mapped_samples = np.zeros_like(lhs_samples)
for i, (variable, dist_info) in enumerate(ranges.items()):
    min_val = dist_info["min"]
    max_val = dist_info["max"]
    mapped_samples[:, i] = uniform.ppf(lhs_samples[:, i], loc=min_val, scale=(max_val - min_val))

In [14]:
samples=pd.DataFrame(mapped_samples)

In [15]:
samples=samples.rename({0:'X1',
                        1:'X2',
                        2:'X3',
                        3:'X4',
                        4:'X5',
                        5:'X6',
                        6:'X7',
                        7:'X8',
                        8:'X9',
                        9:'X10'}, axis=1)

In [16]:
df_samples=np.round(samples, 3)

In [ ]:
print(df_samples['X9'].min())
print(df_samples['X9'].max())

In [17]:
df_samples.to_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\InputsV01.csv", index=False)

In [18]:
df_samples

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
1,7.482,7.247,1.604,0.918,4.087,4.165,17.382,18.326,27.333,26.114
2,7.629,7.381,1.671,0.700,3.949,3.632,18.311,17.273,22.550,24.061
3,5.715,8.797,1.693,0.934,3.572,3.658,18.209,19.397,22.365,22.252
4,6.584,7.771,1.677,0.919,2.684,2.433,19.514,19.038,22.634,24.032
...,...,...,...,...,...,...,...,...,...,...
495,3.986,8.955,1.639,0.301,3.858,3.229,18.474,19.720,22.663,23.256
496,7.572,9.386,1.600,0.056,4.349,4.125,20.456,21.423,26.873,22.475
497,4.182,9.518,1.252,0.932,3.718,2.777,17.911,17.079,25.455,23.669
498,7.640,8.285,1.324,0.948,4.225,2.589,18.303,21.072,27.202,23.023


## Run Automation

In [19]:
df_samples=pd.read_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\InputsV01.csv")

In [20]:
### Creating a parquet scheme using an existing EnergyPlus Result

EP_csv_loc= 'eplusout.csv'
EP_df = pd.read_csv(EP_csv_loc)

# creating the schema
schema = pa.Schema.from_pandas(EP_df)

## We need to add the "Run" column to the schema (We need the "Run" column so later we can match the inputs and outputs)
run_col = pa.field("Run", pa.int64())

new_schema = pa.schema([run_col] + [(field.name, field.type) for field in schema])

# Create a Table with the new schema
table = pa.Table.from_pandas(pd.DataFrame(columns=new_schema.names), schema=new_schema)


# Create the updated schema with the new field
#new_schema = pa.schema([run_col] + schema.fields)

# Assuming you have a Pandas DataFrame 'csv' with the "Run" column
#table = pa.Table.from_pandas(pd.DataFrame(columns=new_schema.names), schema=new_schema)

# Save the table to a Parquet file
pq.write_table(table, 'C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\DatasetV01.parquet')

In [21]:
len(df_samples)

500

In [17]:
idf.idfobjects['Material:NoMass']

[
Material:NoMass,
    CP02 CARPET PAD,          !- Name
    VeryRough,                !- Roughness
    0.2165,                   !- Thermal Resistance
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.8;                      !- Visible Absorptance
, 
Material:NoMass,
    MAT-AIR-WALL,             !- Name
    Rough,                    !- Roughness
    0.2079491,                !- Thermal Resistance
    0.9,                      !- Thermal Absorptance
    0.7;                      !- Solar Absorptance
, 
Material:NoMass,
    NECB_RIns_Wall,           !- Name
    Rough,                    !- Roughness
    3.35,                     !- Thermal Resistance
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
Material:NoMass,
    NECB_RIns_Roof,           !- Name
    Rough,                    !- Roughness
    3.46,              

In [18]:
idf.idfobjects['Schedule:Compact'][-9]


Schedule:Compact,
    NECB_HTGSETP_SCH,         !- Name
    Temperature,              !- Schedule Type Limits Name
    Through: 12/31,           !- Field 1
    For: Weekdays,            !- Field 2
    Until: 06:00,             !- Field 3
    18,                       !- Field 4
    Until: 07:00,             !- Field 5
    20.0,                     !- Field 6
    Until: 21:00,             !- Field 7
    22,                       !- Field 8
    Until: 24:00,             !- Field 9
    18,                       !- Field 10
    For SummerDesignDay,      !- Field 11
    Until: 24:00,             !- Field 12
    15.6,                     !- Field 13
    For: WinterDesignDay,     !- Field 14
    Until: 24:00,             !- Field 15
    21.0,                     !- Field 16
    For: Saturday Sunday AllOtherDays,    !- Field 17
    Until: 24:00,             !- Field 18
    18;                       !- Field 19

In [23]:
### Read the created Parquet table 
data_table = pq.read_table('C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\DatasetV01.parquet') ## For hourly data
start_time=time.time()
annual_results = []

for i in range(len(df_samples)):
    X1=df_samples.loc[i,'X1'] ## Wall Insulation R_value
    X2=df_samples.loc[i,'X2'] ## Roof Insulation R_value
    X3=df_samples.loc[i,'X3'] ## Window U_value
    X4=df_samples.loc[i,'X4'] ## SHGC
    X5=df_samples.loc[i,'X5'] ## COP-High Speed
    X6=df_samples.loc[i,'X6'] ## COP-Low Speed
    X7=df_samples.loc[i,'X7'] ## Occupied Heating SP
    X8=df_samples.loc[i,'X8'] ## UNOccupied Heating SP
    X9=df_samples.loc[i,'X9'] ## Occupied Cooling SP
    X10=df_samples.loc[i,'X10'] ## UNOccupied Cooling SP

    idf.idfobjects['Material:NoMass'][-3].Thermal_Resistance=X1
    idf.idfobjects['Material:NoMass'][-1].Thermal_Resistance=X2
    idf.idfobjects['WindowMaterial:SimpleGlazingSystem'][0].UFactor=X3
    idf.idfobjects['WindowMaterial:SimpleGlazingSystem'][0].Solar_Heat_Gain_Coefficient=X4
    
    for coil in idf.idfobjects['Coil:Cooling:DX:TwoSpeed']:
        coil.High_Speed_Gross_Rated_Cooling_COP=X5
        coil.Low_Speed_Gross_Rated_Cooling_COP=X6
        
    idf.idfobjects['Schedule:Compact'][-9].Field_8=X7 ## Heating SP: 7-21
    idf.idfobjects['Schedule:Compact'][-9].Field_10=X8 ## Heating SP: 21-24
    idf.idfobjects['Schedule:Compact'][-9].Field_4=X8 ## Heating SP: 24-6
    idf.idfobjects['Schedule:Compact'][-10].Field_16=X9 ## Cooling SP 6-21
    idf.idfobjects['Schedule:Compact'][-10].Field_18= X10 ## Cooling SP 21-24
    idf.idfobjects['Schedule:Compact'][-10].Field_14= X10 ## Cooling SP 24-6
    
    idf.run()
    
    ## For hourly Electricity 
    file_path='eplusout.csv'
    csv=pd.read_csv('eplusout.csv')
    run_num = i  ## Run Number_used later to merge inputs and outputs
    csv['Run']=run_num
    
    record_batch = pa.RecordBatch.from_pandas(csv, schema=new_schema)

    # Append the new record batch to the existing table
    data_table = pa.concat_tables([data_table, pa.Table.from_pandas(csv, schema=new_schema)])

    # Save the updated table to the Parquet file
    pq.write_table(data_table,'C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\DatasetV01.parquet')
    
    
    #Read annual data from eplusouttbl.csv
    file_loc = 'eplustbl.csv'
    # Specify the target rows and columns
    targets = {
        "Total_Site_Energy (GJ)": (14, 2),  # C15
        "Total_Natural_Gas (GJ)": (64, 3),  # D65
        "Total_Electricity (GJ)": (64, 2),  #C65
        "Unmet_ASHRAE_55 (hr)": (188, 2),  # C189
        "Peak_Electricity (W)": (323, 2)   # C324
            }
    results = {"Run": i}
#Open the file and process line-by-line
    with open(file_loc, 'r') as file:
        for row_index, line in enumerate(file):
            for key, (target_row, target_column) in targets.items():
                if row_index == target_row:
                    cells = line.split(',')
                    if len(cells) > target_column:
                        value = cells[target_column].strip()
                        results[key] = value
                        print(f"Extracted {key}: {value}")
        
        # Append the results of the current run to the annual results list
        annual_results.append(results)


# Convert results to a DataFrame
annual_results_df = pd.DataFrame(annual_results)

# Save annual results to a CSV file
annual_results_df.to_csv('C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\Annual_V01.csv', index=False)

end_time=time.time()
run_time=end_time-start_time
print(run_time)    


C:\EnergyPlusV23-2-0\energyplus.exe --weather C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\FinalDOE\CAN_ON_OTTAWA-INTL-A_6106001_CWEC.epw --output-directory C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\Notebooks --idd C:\EnergyPlusV23-2-0\Energy+.idd C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\FinalDOE\DOE_V10_4b5472.idf

Extracted Total_Site_Energy (GJ): 1944.07
Extracted Total_Natural_Gas (GJ): 251.32
Extracted Total_Electricity (GJ): 1692.75
Extracted Unmet_ASHRAE_55 (hr): 2452.83
Extracted Peak_Electricity (W): 184351.94

C:\EnergyPlusV23-2-0\energyplus.exe --weather C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\FinalDOE\CAN_ON_OTTAWA-INTL-A_6106001_CWEC.epw --output-directory C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\Notebooks --idd C:\EnergyPlusV23-2-0\Energy+.idd C:\Users\Elin Markarian\OneDrive - Carleton University\Optimization_Seif\Fi